In [1]:
import importlib
import numpy as np
import pandas as pd
import utils

In [ ]:
reload(utils)

In [3]:
from keras.models import Sequential, load_model
from keras.layers import Embedding, LSTM, Dense, Dropout, Flatten, Conv1D
# from keras.optimizers import Adam

Using TensorFlow backend.


In [5]:
emb = utils.load_embedding()

In [6]:
emb.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.363037,0.334529,-0.165212,-0.409537,-0.298907,-0.652681,0.433844,0.339051,-0.419195,0.162223,...,-0.150816,0.380193,0.763373,-0.686894,-0.139185,-0.273024,0.187815,0.011107,0.785204,-0.255765
1,-0.584020,0.390310,0.652820,-0.340300,0.194930,-0.834890,0.119290,-0.572910,-0.568440,0.729890,...,0.285860,-0.052162,-0.508180,-0.634590,0.338890,0.284160,-0.203400,-1.233800,0.467150,0.788580
2,0.257690,0.456290,-0.769740,-0.376790,0.592720,-0.063527,0.205450,-0.573850,-0.290090,-0.136620,...,0.030498,-0.395430,-0.385150,-1.000200,0.087599,-0.310090,-0.346770,-0.314380,0.750040,0.970650
3,-0.039369,1.203600,0.354010,-0.559990,-0.520780,-0.669880,-0.754170,-0.653400,-0.232460,0.586860,...,-0.601410,0.504030,-0.083316,0.202390,0.443000,-0.060769,-0.428070,-0.084135,0.491640,0.085654
4,-0.268200,-0.478700,0.180990,-0.538370,-0.240210,-0.562030,0.209440,0.423580,-0.461470,0.769030,...,0.005399,-0.533440,-0.187060,0.522540,0.243610,0.051387,0.272100,-0.444330,0.019464,0.627820


In [7]:
emb_dim = emb.shape[1]

In [8]:
emb_dim

50

In [5]:
word2ind, ind2word = utils.load_index_word_map()

In [6]:
vocab_size = len(word2ind)

In [8]:
dataset = utils.load_training_samples()

In [10]:
train_x, val_x, train_y, val_y = utils.train_valid_split(dataset.values[:, :-1], dataset.values[:, -1:], test_size=0.3, random_state=43)

In [11]:
train_x.shape

(11174673, 20)

In [12]:
train_y.shape

(11174673, 1)

In [13]:
val_x.shape

(4789145, 20)

In [14]:
val_y.shape

(4789145, 1)

In [15]:
input_length = train_x.shape[1]

In [21]:
model = load_model('model20180126_1.h5')

In [22]:
emb = model.get_weights()[0]

In [23]:
emb_dim = emb.shape[1]

In [28]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=input_length, weights=[emb]))
model.add(Dropout(.2))
model.add(Conv1D(filters=50, kernel_size=4, padding='same', activation='relu'))
model.add(Dropout(.2))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(LSTM(100, activation='relu', return_sequences=True, dropout=.2, recurrent_dropout=.2))
model.add(LSTM(100, activation='relu', dropout=.2, recurrent_dropout=.2))
model.add(Dropout(.2))
# model.add(Flatten())
model.add(Dense(vocab_size, activation='softmax'))

In [24]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 50)            4072600   
_________________________________________________________________
dropout_5 (Dropout)          (None, 20, 50)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 20, 50)            10050     
_________________________________________________________________
dropout_6 (Dropout)          (None, 20, 50)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 20, 32)            6432      
_________________________________________________________________
lstm_5 (LSTM)                (None, 20, 100)           53200     
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               80400     
__________

In [30]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [25]:
model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=2, batch_size=3500)

Train on 11174673 samples, validate on 4789145 samples
Epoch 1/2
11174673/11174673 [==============================] - 1999s - loss: 4.4215 - acc: 0.2527 - val_loss: 4.2056 - val_acc: 0.2718
Epoch 2/2
11174673/11174673 [==============================] - 2027s - loss: 4.3617 - acc: 0.2573 - val_loss: 4.1657 - val_acc: 0.2757


In [26]:
# save model
model.save('model20180126_2.h5')

In [33]:
model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=2, batch_size=3500)

Train on 10972446 samples, validate on 4702476 samples
Epoch 1/2
10972446/10972446 [==============================] - 2129s - loss: 4.2760 - acc: 0.2659 - val_loss: 4.0767 - val_acc: 0.2873
Epoch 2/2
10972446/10972446 [==============================] - 2115s - loss: 4.2726 - acc: 0.2661 - val_loss: 4.0738 - val_acc: 0.2873


In [34]:
# save model
model.save('model20180120_5.h5')

In [37]:
model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=5, batch_size=3500)

Train on 10972446 samples, validate on 4702476 samples
Epoch 1/5
10972446/10972446 [==============================] - 2129s - loss: 4.2568 - acc: 0.2670 - val_loss: 4.0676 - val_acc: 0.2876
Epoch 2/5
10972446/10972446 [==============================] - 2166s - loss: 4.2542 - acc: 0.2670 - val_loss: 4.0639 - val_acc: 0.2885
Epoch 3/5
10972446/10972446 [==============================] - 2154s - loss: 4.2525 - acc: 0.2671 - val_loss: 4.0619 - val_acc: 0.2886
Epoch 4/5
10972446/10972446 [==============================] - 2159s - loss: 4.2502 - acc: 0.2673 - val_loss: 4.0615 - val_acc: 0.2882
Epoch 5/5
10972446/10972446 [==============================] - 2160s - loss: 4.2483 - acc: 0.2673 - val_loss: 4.0613 - val_acc: 0.2884


In [38]:
# save model
model.save('model20180120_7.h5')